In [15]:
import torch

from hqq.core.quantize import *

In [69]:
in_size = 8
out_size = 16

act = torch.arange(1,in_size+1, dtype=torch.float)
kernel = torch.reshape(torch.arange(in_size*out_size, dtype=torch.float), (out_size, in_size))

linear = torch.nn.Linear(in_size, out_size, bias=False)

with torch.no_grad():
    linear.weight.copy_(kernel)

output = linear(act)

quant_config_a0 = BaseQuantizeConfig(nbits=8, group_size=8, quant_zero=False, axis=0)
hqq_linear_a0 = HQQLinear(linear, quant_config_a0, del_orig=False)

quant_config_a1 = BaseQuantizeConfig(nbits=8, group_size=8, quant_zero=False, axis=1)
hqq_linear_a1 = HQQLinear(linear, quant_config_a1, del_orig=False)

#print(f"Base Kernel: {linear.weight.shape}")
#print(f"Quantized Kernel (Axis=0): {hqq_linear_a0.meta['scale'].shape}, {hqq_linear_a0._parameters['W_q'].shape}")
#print(f"Quantized Kernel (Axis=1): {hqq_linear_a1.meta['scale'].shape}, {hqq_linear_a1._parameters['W_q'].shape}")
print(f"Base Kernel: {linear.weight} ({linear.weight.shape})")
print(f"Quantized Kernel (Axis=0):\n Scale: {hqq_linear_a0.meta['scale']} ({hqq_linear_a0.meta['scale'].shape})\n Zero: {hqq_linear_a0.meta['zero']} ({hqq_linear_a0.meta['zero'].shape})\n Weight: {hqq_linear_a0._parameters['W_q']} ({hqq_linear_a0._parameters['W_q'].shape})")
print(f"Quantized Kernel (Axis=1):\n Scale: {hqq_linear_a1.meta['scale']} ({hqq_linear_a1.meta['scale'].shape})\n Zero: {hqq_linear_a1.meta['zero']} ({hqq_linear_a1.meta['zero'].shape})\n Weight: {hqq_linear_a1._parameters['W_q']} ({hqq_linear_a1._parameters['W_q'].shape})")

print(f"Dequantized (Axis=0): {Quantizer.dequantize(hqq_linear_a0._parameters['W_q'].data, hqq_linear_a0.meta)}")
print(f"Dequantized (Axis=1): {Quantizer.dequantize(hqq_linear_a1._parameters['W_q'].data, hqq_linear_a1.meta)}")

Base Kernel: Parameter containing:
tensor([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.],
        [  8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.],
        [ 16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.],
        [ 24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.],
        [ 32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.],
        [ 40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.],
        [ 48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.],
        [ 56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.],
        [ 64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.],
        [ 72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.],
        [ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.],
        [ 88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.],
        [ 96.,  97.,  98.,  99., 100., 101., 102., 103.],
        [104., 105., 106., 107., 108., 109., 110., 111.],
        [112., 113., 114., 115., 116., 117., 118., 119.],
        [120., 121., 122., 123., 124., 125., 126., 127.]], requires_grad=True) (torch.Size([16,

In [71]:
linear.weight.data[0]

tensor([0., 1., 2., 3., 4., 5., 6., 7.])